# Exploring Image and Text-to-Image Embeddings

<a target="_blank" href="https://colab.research.google.com/github/impresso/impresso-datalab-notebooks/blob/main/workshop_resources/ws4-embeddings/MultiModal.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

If something doesn't work, you can [report a problem](https://github.com/impresso/impresso-datalab-notebooks/blob/main/reporting-problems.md).

## What is this notebook about?

Impresso has integrated two types of image embeddings:
- Image-only embeddings with DinoV2.
  - This model is especially good for feature extraction: it identifies the visual feature of the elements in an image.
  - I can only take an image as input (as image or url), and returns an embedding which can be compared to all other images in the collection.
- Text-to-image embeddings with Open-Clip.
  - This model is especially good for multimodal uses: it links the image's visuals to the textual reprensentations it knows. It will be able to "read" text on an image.
  - I can only take either text or an image as input (as image or url), and returns an embedding which can be compared to all other images in the collection, as well as to keyphrases. Typically, one can use a keyphrase to look for images.
  

This notebook explores what can be done with the image and text-to-image embeddings and the Impresso API.

In [1]:
!pip install --force-reinstall git+https://github.com/impresso/impresso-py.git@embeddings-search

  Cloning https://github.com/impresso/impresso-py.git (to revision embeddings-search) to /tmp/pip-req-build-ethgnaj_
  Running command git clone --filter=blob:none --quiet https://github.com/impresso/impresso-py.git /tmp/pip-req-build-ethgnaj_
  Running command git checkout -b embeddings-search --track origin/embeddings-search
  Switched to a new branch 'embeddings-search'
  Branch 'embeddings-search' set up to track remote branch 'embeddings-search' from 'origin'.
  Resolved https://github.com/impresso/impresso-py.git to commit 903d1136da964a23b1fd602d523017564e3702af
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62

### Open a working client connection to the Impresso API

Requires a valid API key and secret. You can obtain these from the Impresso website.


In [1]:
from impresso import connect

impresso = connect('https://dev.impresso-project.ch/public-api/v1')


Click on the following link to access the login page: https://dev.impresso-project.ch/datalab/token
 - 🔤 Enter your email/password on this page.
 - 🔑 Once logged in, a secret token will be generated for you.
 - 📋 Copy this token and paste it into the input field below. Then press "Enter". 👇🏼.

🔑 Enter your token: ··········
🎉 You are now connected to the Impresso API!  🎉
🔗 Using API: https://dev.impresso-project.ch/public-api/v1


During this notebook, we will want to go back and fourth between the notebook and the Impresso App. Having a small helper function to construct links can be really helpful

In [2]:
# we can not see the images, but can access them in the Impresso webapp
# There are two modes "issue" (see the entire issue) or "images" see only the image element

def img_webapp_url(uid, issue_mode=True):
  mode = "issue" if issue_mode else "images"
  pre, suf = uid.split('-a-')
  suffix = f"{pre}-a/view?articleId={suf}" if issue_mode else uid
  return f'https://dev.impresso-project.ch/app/{mode}/{suffix}'

# Case-study: Representations and visuals of Radios

We will explore how radio is represented both in images, and in the programs. This will allow us to explore the image and textual elements using both types of embeddings.

## 1. Text-to-Image embeddings with Open-Clip

First, we want to explore and have a first idea of what are the most similar results to some keywords or phrases mentioning radio.

### a. Keywork search on image captions

A first way to search through the image data (and see the variety of results based on the approach and query), is to do keyword search on our image captions

In [3]:
# start with a small and basic keyword
kw_radio = 'radio'

# first start with keyword search - this will look through the image captions
result = impresso.images.find(term=kw_radio)
result

,issueUid,previewImage,date,caption,pageNumbers,mediaSourceRef.uid,mediaSourceRef.name,mediaSourceRef.type,imageTypes.visualContent,imageTypes.visualContentType,previewUrl,contentItemUid
uid,,,,,,,,,,,,
oeuvre-1944-07-15-a-i0076,oeuvre-1944-07-15-a,,1944-07-15,RADIO-PARIS,[3],oeuvre,L'Œuvre (Paris),newspaper,Not an Image,nan,"https://gallica.bnf.fr/iiif/ark:/12148/bpt6k4622965v/f3/4925,8756,747,153/max/0/default.jpg",nan
oeuvre-1944-08-12-a-i0054,oeuvre-1944-08-12-a,,1944-08-12,RADIO-PARIS,[2],oeuvre,L'Œuvre (Paris),newspaper,Not an Image,nan,"https://gallica.bnf.fr/iiif/ark:/12148/bpt6k4622988v/f2/5305,7413,758,168/max/0/default.jpg",nan
oeuvre-1929-01-03-a-i0134,oeuvre-1929-01-03-a,,1929-01-03,Radio-paris,[8],oeuvre,L'Œuvre (Paris),newspaper,Image,Non-Figurative Visual Content,"https://gallica.bnf.fr/iiif/ark:/12148/bpt6k4617832d/f8/4892,1942,885,550/max/0/default.jpg",nan


### b. Embedding the keyword with open-clip for text-to-image similarity search

Now we will leverage the text-to-image similiraty search enabled by Open-Clip by obtaining the embedding for the same exact keyword and then search through the Open-Clip embeddings of images

In [7]:
# now embed the keyword prompt using the open-clip model
kw_embedding = impresso.tools.embed_text(text=kw_radio, target='multimodal')
kw_embedding

'openclip-768:1zuRvAvlzLvJJxw9tRKdO5yyaryrbIK9wuLOvDO12bz3ezo9+5ogPaXcCj2bbCm9IogYPOXjBz7Sfvq8xCccvbjvbD1F2iW7ZQ0sO5CEf7wH7Qc9HWC9OzApX7y8ThO8DDckPH6dhbzIxBi82Z5RPCbzerwiIo48XpKUvDHtHTxUjcA8QgXJPDXBBzyDrhY7L78DPRhOhLviam487tW/PMJyX7sz6q68v+MnvMDEqTzdOHG8yJnkuz7OEr1EjZw8CFgMPS6dYryzMFm8xeU4PMxlPDwQw1S7eOjDOnfO3bwdgtc7LYsDPe+Nwrtt6Sq8fwOsuQwTG73vtTk7KE7Fu7nfU73P+UY805+dPJ83G7x5Xha9Tc+HPJdgijuPY5i7ks6fPAq1Db0ZFzO7ElLTPH09AD0S11Y6QIPxPDrNYbxr5W49h0ROPJPSo7w6D968ojfHO0MknbwNthM9wETYPHILhrlRnIY7oQgBvUhytTw+knU79xI0vUDxZ7x68IQ88KJUPC/i9bsW3Qq9og6GPNra7Lzj34Y93LM2u/jz1DsP1Ci9u4PRvMz+ar00TmK8PFYjvLRAuTyRy2g8a2dWvIldPjw+M5C7qzV7PJAJgrwlr648xUYBvTjmzDsGqCE8Z7kqPSygbTrLL6I8b2jnPNTtMjzkhAe8HjbRPKM6Ar3ZUlG8hgjyO/x+nbyPkhO894XCPFVBtTtRfrc8HXLXvFtcwrwvFig9Ih4kva8OXL16lnc7YBtzPOVVyTxvxwo79CsEPTe85jyLgLO84b1+O7IkNr1lroa89P5uPeW4kDy7Joe7Pkr5vNmWHjtEYJi8BqypPN4AMzxaats8mJBFvHK39rwDUzS9uDwUua7iIjzYUco8ym26vEPG5bw8VKq88mqRPM1cHjyB0VO9e9GkvIccWTwW5ai8je+pvCTcnzy2t0I8KbndPOLVq7xpAfs8D0aNvDValDyD+yC83DgoPXwU6L

Looking at the embedding above, we can see that it's indeed an open-clip embedding! We only see a bunch of jibbrish but it's actually a base-64 encoded vector of 768 floating point values!

In [57]:
# look at images similar to the embedding
results = impresso.images.find(
  embedding=kw_embedding,
  limit=6
)
results

,issueUid,previewImage,date,contentItemUid,pageNumbers,mediaSourceRef.uid,mediaSourceRef.name,mediaSourceRef.type,imageTypes.visualContent,previewUrl,imageTypes.visualContentType
uid,,,,,,,,,,,
EXP-2009-01-06-a-i0096,EXP-2009-01-06-a,,2009-01-06,EXP-2009-01-06-a-i0088,[12],EXP,L'Express,newspaper,Image,"https://dev.impresso-project.ch/media/iiif/EXP-2009-01-06-a-p0012/97,236,195,200/max/0/default.jpg",Object
IMP-2009-01-06-a-i0088,IMP-2009-01-06-a,,2009-01-06,IMP-2009-01-06-a-i0080,[12],IMP,L'Impartial,newspaper,Image,"https://dev.impresso-project.ch/media/iiif/IMP-2009-01-06-a-p0012/106,242,183,196/max/0/default.jpg",Object
JDG-1995-03-31-a-i0219,JDG-1995-03-31-a,,1995-03-31,nan,[35],JDG,Journal de Genève,newspaper,Image,"https://dev.impresso-project.ch/media/iiif/JDG-1995-03-31-a-p0035/2325,290,248,258/max/0/default.jpg",Object


In [22]:
results.df[['contentItemUid', 'imageTypes.visualContentType']].index

Index(['EXP-2009-01-06-a-i0096', 'IMP-2009-01-06-a-i0088',
       'JDG-1995-03-31-a-i0219', 'EXP-1960-03-31-a-i0096',
       'JDG-1995-10-06-a-i0264', 'EXP-1958-06-11-a-i0118'],
      dtype='object', name='uid')

In [62]:
import numpy as np
import pandas as pd
# Print the URLs for the first 5 images

for uid, r in results.df.head(5).iterrows():
  print(f"Result {uid} - link to image CI {r.previewUrl} - type {r['imageTypes.visualContentType']}")
  if str(r.contentItemUid)!='nan':
    print(f"       {r.contentItemUid} - link to corresponding CI {img_webapp_url(r.contentItemUid)}")

Result EXP-2009-01-06-a-i0096 - link to image CI https://dev.impresso-project.ch/app/images/EXP-2009-01-06-a-i0096 - type Object
       EXP-2009-01-06-a-i0088 - link to corresponding CI https://dev.impresso-project.ch/app/issue/EXP-2009-01-06-a/view?articleId=i0088
Result IMP-2009-01-06-a-i0088 - link to image CI https://dev.impresso-project.ch/app/images/IMP-2009-01-06-a-i0088 - type Object
       IMP-2009-01-06-a-i0080 - link to corresponding CI https://dev.impresso-project.ch/app/issue/IMP-2009-01-06-a/view?articleId=i0080
Result JDG-1995-03-31-a-i0219 - link to image CI https://dev.impresso-project.ch/app/images/JDG-1995-03-31-a-i0219 - type Object
Result EXP-1960-03-31-a-i0096 - link to image CI https://dev.impresso-project.ch/app/images/EXP-1960-03-31-a-i0096 - type Non-Figurative Visual Content
       EXP-1960-03-31-a-i0087 - link to corresponding CI https://dev.impresso-project.ch/app/issue/EXP-1960-03-31-a/view?articleId=i0087
Result JDG-1995-10-06-a-i0264 - link to image CI h

As one can see, the extracted images either feature iillustrations of radios (physical radio sets) or illustrated headers of radio sections.

We can try to filter by image type (between `Object`, `Non-Figurative Visual Content` and `Ornament or Illustrated Title`)


In [63]:
# filter results based on the image content type
object_results = impresso.images.find(
  content_type="Object",
  embedding=kw_embedding,
  limit=5
)

# Print the URLs for the first 5 images
print(f"Results for images of type Object")
for uid, r in object_results.df.head(5).iterrows():
  print(f"Result {uid} - link to image CI {img_webapp_url(uid, issue_mode=False)}")
  if str(r.contentItemUid)!='nan':
    print(f"       {r.contentItemUid} - link to corresponding CI {img_webapp_url(r.contentItemUid)}")

Results for images of type Object
Result EXP-2009-01-06-a-i0096 - link to image CI https://dev.impresso-project.ch/app/images/EXP-2009-01-06-a-i0096
       EXP-2009-01-06-a-i0088 - link to corresponding CI https://dev.impresso-project.ch/app/issue/EXP-2009-01-06-a/view?articleId=i0088
Result IMP-2009-01-06-a-i0088 - link to image CI https://dev.impresso-project.ch/app/images/IMP-2009-01-06-a-i0088
       IMP-2009-01-06-a-i0080 - link to corresponding CI https://dev.impresso-project.ch/app/issue/IMP-2009-01-06-a/view?articleId=i0080
Result JDG-1995-10-06-a-i0264 - link to image CI https://dev.impresso-project.ch/app/images/JDG-1995-10-06-a-i0264
       JDG-1995-10-06-a-i0257 - link to corresponding CI https://dev.impresso-project.ch/app/issue/JDG-1995-10-06-a/view?articleId=i0257
Result IMP-2010-02-02-a-i0126 - link to image CI https://dev.impresso-project.ch/app/images/IMP-2010-02-02-a-i0126
       IMP-2010-02-02-a-i0120 - link to corresponding CI https://dev.impresso-project.ch/app/is

As we can see we're finding back the elements from the previous search, without the image `EXP-1960-03-31-a-i0096`, but other ones instead

We also see that the image used in EXP-2009-01-06-a-i0096 and IMP-2009-01-06-a-i0080 was then reused a few weeks lated in IMP-2010-02-02-a-i0120 by the editors!

In [64]:
# filter results based on the image content type
non_fig_results = impresso.images.find(
  content_type="Non-Figurative Visual Content",
  embedding=kw_embedding,
  limit=5
)

# Print the URLs for the first 5 images
print(f"Results for images of type Non-Figurative Visual Content")
for uid, r in non_fig_results.df.head(5).iterrows():
  print(f"Result {uid} - link to image CI {img_webapp_url(uid, issue_mode=False)}")
  if str(r.contentItemUid)!='nan':
    print(f"       {r.contentItemUid} - link to corresponding CI {img_webapp_url(r.contentItemUid)}")

Results for images of type Non-Figurative Visual Content
Result EXP-1960-03-31-a-i0096 - link to image CI https://dev.impresso-project.ch/app/images/EXP-1960-03-31-a-i0096
       EXP-1960-03-31-a-i0087 - link to corresponding CI https://dev.impresso-project.ch/app/issue/EXP-1960-03-31-a/view?articleId=i0087
Result EXP-1958-06-11-a-i0118 - link to image CI https://dev.impresso-project.ch/app/images/EXP-1958-06-11-a-i0118
       EXP-1958-06-11-a-i0114 - link to corresponding CI https://dev.impresso-project.ch/app/issue/EXP-1958-06-11-a/view?articleId=i0114


We can see here there are much less results of this type which is less frequent in general in the data. However, in both occasions it's the Radio section logo which was identified by the model, since it's also text-based.

In [65]:
# filter results based on the image content type
title_results = impresso.images.find(
  content_type="Ornament or Illustrated Title",
  embedding=kw_embedding,
  limit=5
)

# Print the URLs for the first 5 images
print(f"Results for images of type Ornament or Illustrated Title")
for uid, r in title_results.df.head(5).iterrows():
  print(f"Result {uid} - link to image CI {img_webapp_url(uid, issue_mode=False)}")
  if str(r.contentItemUid)!='nan':
    print(f"       {r.contentItemUid} - link to corresponding CI {img_webapp_url(r.contentItemUid)}")

Results for images of type Ornament or Illustrated Title


No results! TODO remove maybe or analyse.

### c. Complexifying the search

Now we want to look for more specific things: the radio program of a newspaper. We can try to look for this with a more descriptive query which we will embed again

In [66]:
program_query = "Weekly radio program"

# now embed the keyword prompt using the open-clip model
pgm_embedding = impresso.tools.embed_text(text=program_query, target='multimodal')
pgm_embedding


# look at images similar to the embedding
pgm_results = impresso.images.find(
  embedding=pgm_embedding,
  limit=6
)

for uid, r in pgm_results.df.head(5).iterrows():
  print(f"Result {uid} - link to image CI {img_webapp_url(uid, issue_mode=False)} - type {r['imageTypes.visualContentType']}")
  if str(r.contentItemUid)!='nan':
    print(f"       {r.contentItemUid} - link to corresponding CI {img_webapp_url(r.contentItemUid)}")

Result IMP-1938-09-08-a-i0065 - link to image CI https://dev.impresso-project.ch/app/images/IMP-1938-09-08-a-i0065 - type Ornament or Illustrated Title
       IMP-1938-09-08-a-i0061 - link to corresponding CI https://dev.impresso-project.ch/app/issue/IMP-1938-09-08-a/view?articleId=i0061
Result IMP-1938-01-24-a-i0046 - link to image CI https://dev.impresso-project.ch/app/images/IMP-1938-01-24-a-i0046 - type Ornament or Illustrated Title
       IMP-1938-01-24-a-i0043 - link to corresponding CI https://dev.impresso-project.ch/app/issue/IMP-1938-01-24-a/view?articleId=i0043
Result IMP-1941-10-11-a-i0120 - link to image CI https://dev.impresso-project.ch/app/images/IMP-1941-10-11-a-i0120 - type Ornament or Illustrated Title
       IMP-1941-10-11-a-i0117 - link to corresponding CI https://dev.impresso-project.ch/app/issue/IMP-1941-10-11-a/view?articleId=i0117
Result IMP-1938-09-10-a-i0064 - link to image CI https://dev.impresso-project.ch/app/images/IMP-1938-09-10-a-i0064 - type Ornament or

We have found our illustrated section titles!
It appears that this query has allowed us to fetch many of the radio programs in L'Impartial in the late 30's and early 40's.

Clip is multi-lingual so we could try with a query in french

In [67]:
program_query_fr = "Programme Radio de la semaine"

# now embed the keyword prompt using the open-clip model
pgm_fr_embedding = impresso.tools.embed_text(text=program_query_fr, target='multimodal')
pgm_fr_embedding


# look at images similar to the embedding
pgm_fr_results = impresso.images.find(
  embedding=pgm_fr_embedding,
  limit=6
)

for uid, r in pgm_fr_results.df.head(5).iterrows():
  print(f"Result {uid} - link to image CI {img_webapp_url(uid, issue_mode=False)} - type {r['imageTypes.visualContentType']}")
  if str(r.contentItemUid)!='nan':
    print(f"       {r.contentItemUid} - link to corresponding CI {img_webapp_url(r.contentItemUid)}")

Result IMP-1980-05-24-a-i0269 - link to image CI https://dev.impresso-project.ch/app/images/IMP-1980-05-24-a-i0269 - type Graph
       IMP-1980-05-24-a-i0266 - link to corresponding CI https://dev.impresso-project.ch/app/issue/IMP-1980-05-24-a/view?articleId=i0266
Result IMP-1980-03-25-a-i0230 - link to image CI https://dev.impresso-project.ch/app/images/IMP-1980-03-25-a-i0230 - type Graph
       IMP-1980-03-25-a-i0229 - link to corresponding CI https://dev.impresso-project.ch/app/issue/IMP-1980-03-25-a/view?articleId=i0229
Result IMP-1981-03-04-a-i0226 - link to image CI https://dev.impresso-project.ch/app/images/IMP-1981-03-04-a-i0226 - type nan
       IMP-1981-03-04-a-i0225 - link to corresponding CI https://dev.impresso-project.ch/app/issue/IMP-1981-03-04-a/view?articleId=i0225
Result IMP-1980-04-05-a-i0232 - link to image CI https://dev.impresso-project.ch/app/images/IMP-1980-04-05-a-i0232 - type Graph
       IMP-1980-04-05-a-i0231 - link to corresponding CI https://dev.impresso-p

We now have a very similar case scenario, with results being programs, but this time more recent occurences, and of TV programs. (Note that the swiss national radio and TV channel have the same name!)

It's interesting to see we can land easily on programs, but what about really finding images of radio stations, maybe with people listening to it?

In [71]:
radio_query_fr = "Personnes écoutant la radio à côté du poste de radio."

# now embed the keyword prompt using the open-clip model
radio_fr_embedding = impresso.tools.embed_text(text=radio_query_fr, target='multimodal')
radio_fr_embedding


# look at images similar to the embedding
radio_fr_results = impresso.images.find(
  embedding=radio_fr_embedding,
  limit=6
)

for uid, r in radio_fr_results.df.head(5).iterrows():

  print(f"Result {uid} - link to image CI {r.previewUrl} - type {r['imageTypes.visualContentType']}")
  if 'contentItemUid' in r and str(r.contentItemUid)!='nan':
    print(f"       {r.contentItemUid} - link to corresponding CI {img_webapp_url(r.contentItemUid)}")

Result LLE-1952-10-25-a-i0236 - link to image CI https://dev.impresso-project.ch/media/iiif/LLE-1952-10-25-a-p0017/237,3718,1114,668/max/0/default.jpg - type Non-Figurative Visual Content
Result LLE-1952-11-15-a-i0300 - link to image CI https://dev.impresso-project.ch/media/iiif/LLE-1952-11-15-a-p0023/2619,186,1109,665/max/0/default.jpg - type Non-Figurative Visual Content
Result oeuvre-1935-04-05-a-i0187 - link to image CI https://gallica.bnf.fr/iiif/ark:/12148/bpt6k46197689/f8/2299,595,838,608/max/0/default.jpg - type Object
Result lepetitparisien-1941-03-15-a-i0082 - link to image CI https://gallica.bnf.fr/iiif/ark:/12148/bpt6k684309k/f1/69,1788,815,1211/max/0/default.jpg - type Human Representation - Scene
Result oeuvre-1938-11-04-a-i0187 - link to image CI https://gallica.bnf.fr/iiif/ark:/12148/bpt6k4621627t/f7/4642,3797,911,1083/max/0/default.jpg - type Non-Figurative Visual Content


Here we directly find more images of real radio stations, and sometimes also with people. It can be interesting to compare this result to a similar english sentence, or to try to specify the type to include a human representation

In [72]:
radio_query_en = "People listening to a radio monitor."

# now embed the keyword prompt using the open-clip model
radio_en_embedding = impresso.tools.embed_text(text=radio_query_en, target='multimodal')
radio_en_embedding


# look at images similar to the embedding
radio_en_results = impresso.images.find(
  embedding=radio_en_embedding,
  limit=6
)

for uid, r in radio_en_results.df.head(5).iterrows():

  print(f"Result {uid} - link to image CI {r.previewUrl} - type {r['imageTypes.visualContentType']}")
  if 'contentItemUid' in r and str(r.contentItemUid)!='nan':
    print(f"       {r.contentItemUid} - link to corresponding CI {img_webapp_url(r.contentItemUid)}")

Result IMP-1997-07-21-a-i0042 - link to image CI https://dev.impresso-project.ch/media/iiif/IMP-1997-07-21-a-p0004/1267,690,1710,1149/max/0/default.jpg - type Human Representation - Scene
       IMP-1997-07-21-a-i0038 - link to corresponding CI https://dev.impresso-project.ch/app/issue/IMP-1997-07-21-a/view?articleId=i0038
Result tageblatt-1941-02-15-a-i0113 - link to image CI https://iiif.eluxemburgensia.lu/image/iiif/2/ark:70795%2fgwq95j%2fpages%2f12/2913,229,794,1027/max/0/default.jpg - type Human Representation - Scene
Result VHT-1979-06-20-a-i0035 - link to image CI https://dev.impresso-project.ch/media/iiif/VHT-1979-06-20-a-p0009/1520,1779,1175,812/max/0/default.jpg - type Human Representation - Scene
Result EXP-1983-08-31-a-i0208 - link to image CI https://dev.impresso-project.ch/media/iiif/EXP-1983-08-31-a-p0010/521,693,849,454/max/0/default.jpg - type Human Representation - Scene
       EXP-1983-08-31-a-i0195 - link to corresponding CI https://dev.impresso-project.ch/app/issue

Having a query in english seems to have done the trick here; as you can see most images are of the type human representation!

Don't hesitate to explore further with different queries, more complex and simple ones, varying languages and using the help of the image type filter to specify more precisely what's of interest!

## 2. Image-only embeddings with DinoV2

Now that we've looked a bit into the illustrations of radio images and radio programs, we'll dive more into image-to-image embeddings, and try to recover images wich are very similar to ones of specific interest to us.

### a. Similarities within impresso - from an image we have identified

The first example will be for an image from the Impresso collection which we have identified to be of interest.
Imagine for instance that we are looking to work on the rise of access to new technologies in the 80's. Then maybe the image [EXP-1983-08-31-a-i0208](https://dev.impresso-project.ch/media/iiif/EXP-1967-03-29-a-p0003/497,637,824,564/max/0/default.jpg) from content-item [EXP-1983-08-31-a-i0195](https://dev.impresso-project.ch/app/issue/EXP-1983-08-31-a/view?articleId=i0195) would beinteresting to take as a basis to find other, similar articles

In [74]:
example_image_id = 'EXP-1983-08-31-a-i0208'

# since the image is already in Impresso, we can get its embeddings very easily with:
example_embedding = impresso.images.get_embeddings(example_image_id)
# the first element is Openclip, the second the dino embedding
example_embedding[1]

'dinov2-1024:g2p8vPJhgzu3OYQ8xNlnOl9Y/7yLUXy8ilwMO41QibztvAG7P4VLvD6UpjzlDA09m3b3vKnwWD0n3RY8wgWvOt2mQT0bFLC99ts5vEMeET26t5w9Kan/PDX/MT2HbZ48GZievMIdUbze90g82RkRvGKzBL3vCxg82M7SO2mSbDzja189NDnsvJpeDbxVBCG7ghYQPSGZJTwI1Ne7d6xNPSDFBLzafuO7bYOpPNxI2bwM4tE8F8+zPKHTcb3rOC49yzgOOrOkvTyiRSO88tqHPOq887xNct27zMjDu7ADGr3qQBU8B5mFPcdh2bxoYCQ9rBXfPL2JHr1PZ8y8jEKGPNRRFTwXZlG8INDdPLNMtTyVA9e8/XajPWMwOb1gW+Q8G4gVPetcA7yOdbA8NC3aPJbqQrz1G4a80SOoPB1rcj31FnG9w1YNvS6M3jwFSFW7xc1LPcdcAjwFrq86TYDUuy2ikb3JBKw8meAxO5SMFLwje1U8OvgGPVCC9zwdAPA83A4YO1FIAj2mLGq9NbUDPZgOVTx6Yq68EDi+O+hA2j2v0hC93qTdPEBr5jyRc5M7CsOgvCulXT2DYuY8hW+wOznEiDsGH6q8L2qoPMp+GLywICM8V7lmvQ/KsrzOmWO9hGwLvWrTOb0v68+8GSGXu0dpE70TD5U754uivGcqKT1tSQs9C4rsvIaM2DwiPci8Ge77vHEdl7nKpeo89COFvfKYfb3Sjxg9B/FGPQJG7Dxth2q8TzZovadgEb23k5W8m6HYuxWDbz1SZCs9ACQRPOikN7xMl968mscMvaZpLLzqhSi9t1dMPWo1j7w0xbw8D2OZPQh1CLxYGwK9DsE+PFY2wDyWqMm8Dzl1vCR6eroy7sI8CqTzO8EQ5DxhA3U6wA6fPIBBH7yiz4I627ZKvPZxEjwewDO9BICivPc+pjx+Xyq7LXTpO2iX7zyU5na7VE4CvX9UATw7RgG9RLS3OndcLb1

In [75]:
# look at images similar to the embedding
dino_results = impresso.images.find(
  embedding=example_embedding[1],
  limit=6
)

for uid, r in dino_results.df.head(5).iterrows():

  print(f"Result {uid} - link to image CI {r.previewUrl} - type {r['imageTypes.visualContentType']}")
  if 'contentItemUid' in r and str(r.contentItemUid)!='nan':
    print(f"       {r.contentItemUid} - link to corresponding CI {img_webapp_url(r.contentItemUid)}")

Result EXP-1983-08-31-a-i0208 - link to image CI https://dev.impresso-project.ch/media/iiif/EXP-1983-08-31-a-p0010/521,693,849,454/max/0/default.jpg - type Human Representation - Scene
       EXP-1983-08-31-a-i0195 - link to corresponding CI https://dev.impresso-project.ch/app/issue/EXP-1983-08-31-a/view?articleId=i0195
Result EXP-1969-09-04-a-i0031 - link to image CI https://dev.impresso-project.ch/media/iiif/EXP-1969-09-04-a-p0002/317,665,557,361/max/0/default.jpg - type Human Representation - Scene
       EXP-1969-09-04-a-i0021 - link to corresponding CI https://dev.impresso-project.ch/app/issue/EXP-1969-09-04-a/view?articleId=i0021
Result VHT-1976-04-07-a-i0023 - link to image CI https://dev.impresso-project.ch/media/iiif/VHT-1976-04-07-a-p0005/2135,3109,1335,935/max/0/default.jpg - type Human Representation - Scene
Result luxland-2007-02-09-a-i0136 - link to image CI https://iiif.eluxemburgensia.lu/image/iiif/2/ark:70795%2fx5v85z%2fpages%2f24/2554,2782,975,643/max/0/default.jpg - 

In [ ]:
# use a wikipedia image url as starting point
image_url = '/content/Dioecesis_Orientis_400_AD.png'
embedding = impresso.tools.embed_image(image=image_url, target="image")


In [ ]:
# look at images similar to the embedding
results = impresso.images.find(
  embedding=embedding,
  limit=5
)

In [ ]:
results

,issueUid,previewImage,date,contentItemUid,pageNumbers,mediaSourceRef.uid,mediaSourceRef.name,mediaSourceRef.type,imageTypes.visualContent,previewUrl,imageTypes.visualContentType
uid,,,,,,,,,,,
EXP-1999-07-24-a-i0060,EXP-1999-07-24-a,,1999-07-24,EXP-1999-07-24-a-i0056,[5],EXP,L'Express,newspaper,Image,https://dev.impresso-project.ch/assets/images/not-allowed.png,Map - Geopolitical
IMP-1998-04-06-a-i0035,IMP-1998-04-06-a,,1998-04-06,IMP-1998-04-06-a-i0032,[3],IMP,L'Impartial,newspaper,Image,https://dev.impresso-project.ch/assets/images/not-allowed.png,Map - Weather
EXP-1998-04-06-a-i0033,EXP-1998-04-06-a,,1998-04-06,EXP-1998-04-06-a-i0029,[3],EXP,L'Express,newspaper,Image,https://dev.impresso-project.ch/assets/images/not-allowed.png,Map - Weather


In [ ]:
# we can not see the images, but can acccess them in the Impresso webapp
# but have to create a link
def img_webapp_url(uid):
  pre, suf = uid.split('-a-')
  return f'https://dev.impresso-project.ch/app/issue/{pre}-a/view?articleId={suf}'

In [ ]:
# add Impresso webapp link
df = results.df.copy()
df['webapp_url'] = df.contentItemUid.apply(img_webapp_url)

In [ ]:
df.webapp_url.values

In [ ]:
# filter results based on the image content type
results = impresso.images.find(
  content_type="Map - Geopolitical",
  embedding=embedding,
  limit=5
)

In [ ]:
# inspect in Impresso webapp
df = results.df.copy()
df['webapp_url'] = df.contentItemUid.apply(img_webapp_url)
df['webapp_url'].values

In [ ]:
# filter with a date range
from impresso import DateRange

results = impresso.images.find(
  content_type="Map - Geopolitical",
  embedding=embedding,
  date_range=DateRange("1800-01-01", "1960-01-01"),
  limit=5
)

results


In [ ]:
# use an images prompt for find geopolitical maps
result = impresso.images.find(term="Palästina",content_type="Map - Geopolitical")
result

In [ ]:
# get the image embeddings
image_id = 'tageblatt-1937-07-10-a-i0080'
embedding = impresso.images.get_embeddings(image_id)
embedding

In [ ]:
# use the embedded image to find similar items
results = impresso.images.find(
  content_type="Map - Geopolitical",
  embedding=embedding[0],
  date_range=DateRange("1800-01-01", "1960-01-01"),
  limit=5
)

In [ ]:
results

In [ ]:
# combined text-to-image search with a date range
result = impresso.images.find(term="Jerusalem", date_range=DateRange("1900-01-01", "1960-01-01"))
result

In [ ]:
results.df['imageTypes.visualContent']

In [ ]:
# trying to find other representations
result = impresso.images.find(term="Jerusalem",
                             date_range=DateRange("1900-01-01", "1960-01-01"),
                             content_type=OR('Scenery or Landscape','Human representations - Scene'))
result

# Fin.